# Dataset cleaning script

Does few steps:

1. Imports the CSV
2. Removes the regions from dataset
3. Moves format to long format, as we can easily drop missing (N/A) rows
4. Get's the features to include into training
5. Then creates dataset ready for training (with the dependent variable as index and independent variables as features)

# 1 Import CSV (exported from Excel to CSV)

In [1]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

file_path = "WDIEXCEL.xlsx"

# Load the data from the CSV file
data = pd.read_excel(file_path)

# 2 Remove regions (and unused years from dataset)

In [2]:
countries_to_keep = [
  "AFG", "ALB", "DZA", "AND", "AGO", "ATG", "ARG", "ARM", "AUS", "AUT", "AZE", "BHS",
  "BHR", "BGD", "BRB", "BLR", "BEL", "BLZ", "BEN", "BTN", "BOL", "BIH", "BWA", "BRA",
  "BRN", "BGR", "BFA", "BDI", "CPV", "KHM", "CMR", "CAN", "CAF", "TCD", "CHL", "CHN",
  "COL", "COM", "COG", "COD", "CRI", "CIV", "HRV", "CUB", "CYP", "CZE", "DNK", "DJI",
  "DMA", "DOM", "ECU", "EGY", "SLV", "GNQ", "ERI", "EST", "SWZ", "ETH", "FJI", "FIN",
  "FRA", "GAB", "GMB", "GEO", "DEU", "GHA", "GRC", "GRD", "GTM", "GIN", "GNB", "GUY",
  "HTI", "HND", "HUN", "ISL", "IND", "IDN", "IRN", "IRQ", "IRL", "ISR", "ITA", "JAM",
  "JPN", "JOR", "KAZ", "KEN", "KIR", "PRK", "KOR", "KWT", "KGZ", "LAO", "LVA", "LBN",
  "LSO", "LBR", "LBY", "LIE", "LTU", "LUX", "MDG", "MWI", "MYS", "MDV", "MLI", "MLT",
  "MHL", "MRT", "MUS", "MEX", "FSM", "MDA", "MCO", "MNG", "MNE", "MAR", "MOZ", "MMR",
  "NAM", "NRU", "NPL", "NLD", "NZL", "NIC", "NER", "NGA", "MKD", "NOR", "OMN", "PAK",
  "PLW", "PAN", "PNG", "PRY", "PER", "PHL", "POL", "PRT", "QAT", "ROU", "RUS", "RWA",
  "KNA", "LCA", "VCT", "WSM", "SMR", "STP", "SAU", "SEN", "SRB", "SYC", "SLE", "SGP",
  "SVK", "SVN", "SLB", "SOM", "ZAF", "SSD", "ESP", "LKA", "SDN", "SUR", "SWE", "CHE",
  "SYR", "TJK", "TZA", "THA", "TLS", "TGO", "TON", "TTO", "TUN", "TUR", "TKM", "TUV",
  "UGA", "UKR", "ARE", "GBR", "USA", "URY", "UZB", "VUT", "VAT", "VEN", "VNM", "YEM",
  "ZMB", "ZWE"

]

# Generate year strings for the columns to drop (1960 untill 2000)
years = [str(year) for year in range(1960, 2001)]

# drop the Country Name, Indicator Name and years 1960-2000 columns
preprocessed_data = data.drop(columns=["Country Name", "Indicator Name"] + years)

# drop the combined country data (world, asia, etc.)
preprocessed_data = preprocessed_data[preprocessed_data["Country Code"].isin(countries_to_keep)]

# 3 Move format to long format (and drop N/A rows)

In [3]:
long_formatted_data = preprocessed_data.melt(
    id_vars=["Country Code", "Indicator Code"],  # Columns to keep fixed
    var_name="Year",                             # Column name for years
    value_name="Value"                           # Column name for indicator values
)
long_formatted_data.dropna(inplace=True)

# 4 Include features

In [41]:
# Separate GDP and Life Expectancy into different columns
long_formatted_data["Year"] = long_formatted_data["Year"].astype(int)  # Ensure Year is numeric

# Get the GDP (still have to convert log to linear)
gdp = long_formatted_data[long_formatted_data["Indicator Code"] == "NY.GDP.PCAP.CD"].rename(columns={"Value": "GDP"})
life_expectancy = long_formatted_data[long_formatted_data["Indicator Code"] == "SP.DYN.LE00.IN"].rename(columns={"Value": "Life Expectancy"})
underfivedeaths = long_formatted_data[long_formatted_data["Indicator Code"] == "SH.DYN.MORT"].rename(columns={"Value": "UnderFiveDeaths"})
adultmortality = long_formatted_data[long_formatted_data["Indicator Code"] == "SP.DYN.AMRT.MA"].rename(columns={"Value": "MortRateAdult"})
# alcohol = long_formatted_data[long_formatted_data["Indicator Code"] == "SH.ALC.PCAP.LI"].rename(columns={"Value": "Alcohol"})
anemia = long_formatted_data[long_formatted_data["Indicator Code"] == "SH.ANM.CHLD.ZS"].rename(columns={"Value": "AnemiaChild"})
healthexpenditure = long_formatted_data[long_formatted_data["Indicator Code"] == "SH.XPD.CHEX.PP.CD"].rename(columns={"Value": "HealthExpenditure"})
cleanfuels = long_formatted_data[long_formatted_data["Indicator Code"] == "EG.CFT.ACCS.ZS"].rename(columns={"Value": "CleanFuels"})
populationcount = long_formatted_data[long_formatted_data["Indicator Code"] == "SP.POP.TOTL"].rename(columns={"Value": "PopulationCount"})
basicsanitation = long_formatted_data[long_formatted_data["Indicator Code"] == "SH.STA.BASS.ZS"].rename(columns={"Value": "BasicSanitation"})
schoolenrollment = long_formatted_data[long_formatted_data["Indicator Code"] == "SE.SEC.NENR.MA"].rename(columns={"Value": "SchoolEnrollment"})
thinness = long_formatted_data[long_formatted_data["Indicator Code"] == "SN.ITK.DEFC.ZS"].rename(columns={"Value": "Thinness"})
features_data = pd.DataFrame()
features_data = pd.merge(
    gdp[["Country Code", "Year", "GDP"]],
    life_expectancy[["Country Code", "Year", "Life Expectancy"]],
    on=["Country Code", "Year"]
)

features_data = pd.merge(
     underfivedeaths[["Country Code", "Year", "UnderFiveDeaths"]],
     features_data,
     on=["Country Code", "Year"]
 )
features_data = pd.merge(
    healthexpenditure[["Country Code", "Year", "HealthExpenditure"]],
    features_data,
    on=["Country Code", "Year"]
)
features_data = pd.merge(
    thinness[["Country Code", "Year", "Thinness"]],
    features_data,
    on=["Country Code", "Year"]
)

features_data = pd.merge(
    adultmortality[["Country Code", "Year", "MortRateAdult"]],
    features_data,
    on=["Country Code", "Year"]
)

features_data = pd.merge(
    cleanfuels[["Country Code", "Year", "CleanFuels"]],
    features_data,
    on=["Country Code", "Year"]
)

# features_data = pd.merge(
#     anemia[["Country Code", "Year", "AnemiaChild"]],
#     features_data,
#     on=["Country Code", "Year"]
# )

features_data = pd.merge(
    populationcount[["Country Code", "Year", "PopulationCount"]],
    features_data,
    on=["Country Code", "Year"]
)

features_data = pd.merge(
    basicsanitation[["Country Code", "Year", "BasicSanitation"]],
    features_data,
    on=["Country Code", "Year"]
)

# features_data = pd.merge(
#     schoolenrollment[["Country Code", "Year", "SchoolEnrollment"]],
#     features_data,
#     on=["Country Code", "Year"]
# )

## 4.5 Transform from log

In [42]:
import numpy as np
features_data["GDP"] = np.log(features_data["GDP"])
features_data["CleanFuels"] = np.log(features_data["CleanFuels"])
features_data["BasicSanitation"] = np.log(features_data["BasicSanitation"])

# 5 Create dataset suitable for training

Training datasets are normally formatted in a way were the index corresponds to the dependent variable.

In [43]:
dataset_final = features_data.set_index("Life Expectancy")

# Step 5: Rearrange columns to include Country, GDP, and Year as features (for now, for the final set: Year and Country Code should be removed!)
dataset_final = dataset_final[["GDP", "MortRateAdult", "CleanFuels", "HealthExpenditure", "PopulationCount", "BasicSanitation", "Thinness"]]		

# Display the final dataset
print(dataset_final)

dataset_final.to_csv("final_dataset.csv")

                       GDP  UnderFiveDeaths  MortRateAdult  CleanFuels  \
Life Expectancy                                                          
75.639000         7.155911             25.8        109.086    3.713572   
70.823000         7.469986             40.1        152.528    4.577799   
46.590000         6.268081            198.9        481.080    3.706228   
74.186000         8.877518             19.0        174.541    4.567468   
70.932000         6.516415             29.2        218.454    4.415220   
...                    ...              ...            ...         ...   
76.980488        11.059246              6.3        162.991    4.605170   
78.430000         9.658258              6.3        137.903    4.605170   
70.331000         7.472676             14.7        211.749    4.421247   
70.299000         7.978570             24.0        186.374    1.987874   
75.378000         8.184889             20.9        163.167    4.550714   

                 HealthExpenditure  P